In [2]:
#################################################################################
#
# Project           : University Courses / Modelagem Matemática em Finanças I
#
# Program name      : binomial.py
#
# Author            : gilmiranda
#
# Date created      : 03-30-2019
#
# Purpose           : Take a binomial one step tree with initial values and two final values
#                     and discretize for an arbitrary time 't'
#
# Revision History  :
#
# Date          Author          Ref    Revision (Date in MM-DD-YYYY format)
# 03-30-2019    gilmiranda      1      Created.
# 03-31-2019    gilmiranda      2      Bug-fix no calc_Vs, agora o algoritmo
#                                      calcula corretamente o V_0, ou valor justo
#
#################################################################################

class binomial:
    def __init__(self):
        self.Sf = [0,0] ## Guarda os valores finais da binomial inicial
        self.Sf_disc = [] ## Guarda os valores finais da multinomial, já discretizada
        self.Vs = [] ## Matriz com todos os Vs ao longo do tempo na árvore, sendo o indíce n de cada vetor, seu t_n
        self.Vf = []
        self.ts = 1 ## Tempo a ser discretizado
        self.s0 = 10 ## Valor inicial da árvore
        self.u = self.Sf[0]/self.s0 ## Up factor inicial
        self.d = self.Sf[1]/self.s0 ## Down factor inicial
        self.r = 1/10 ## Taxa de juros
        self.k = 1 ## Strike da opção call/put
        self.u_disc = 1
        self.d_disc = 1
        self.ptil = 1
        self.qtil = 1

    def set_k(self, k): ## Seta strike
        self.k = k

    def set_Sf(self,s): ## Seta Sf
        self.Sf[0] = s[0]
        self.Sf[1] = s[1]

    def set_s0(self,s): ## Seta S0
        if s == 0: raise ValueError('Preço inicial 0 não será modelado')
        self.s0 = s

    def set_Ts(self,t):
        self.ts = t

    def set_riskneutral(self):
        self.ptil = ((1+self.r)-self.d_disc)/(self.u_disc - self.d_disc)
        self.qtil = 1-self.ptil

    def set_factors(self):
            self.u_disc =((self.Sf[0]/self.s0)**(1/(self.ts-1)))
            self.d_disc = ((self.Sf[1]/self.s0)**(1/(self.ts-1)))

    def discretize(self):
        self.Sf_disc = []
        for i in range(0,self.ts):
            self.Sf_disc.append(self.u_disc**(self.ts-1-i)*self.d_disc**(i)*self.s0)

    def print_Sf(self):
        print(self.Sf_disc)

    def call(self, s):
        if self.k >= s: return 0
        return s - self.k

    def put(self, s):
        if s >= self.k: return 0
        return self.k - s

    def set_Vf(self, f):
        self.Vf = [f(i) for i in self.Sf_disc]

    def calcprev_V(self,Vnxt):
        return (1/(1+self.r))*(Vnxt[0]*self.ptil + Vnxt[1]*self.qtil)

    def calc_Vs(self):
        self.Vs.append(self.Vf)
        for j in range(0,self.ts-1):
            aux = []
            c = len(self.Vs[-1])
            for i in range(0,c-1):
                aux.append(self.calcprev_V((self.Vs[-1][i],self.Vs[-1][i+1])))
            self.Vs.append(aux)
        self.Vs = self.Vs[::-1]


In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [41]:
## S -> array com 3 valores: valor inicial, valor final Heads, valor final Tails
## t -> fator de divisão do tempo
## factors -> array com 3 valores: up, down, r = taxa de juros
## tipo -> 0 = call, 1 = put
## retv0 -> True calcula V_0
def discretizador(S, t, factors = [2.0,0.5,.25], tipo = 0, retv0 = True):
    ## Variavel para armazenar o estado inicial do modelo, S = [S0, S1(H), S1(T)]
    ts = int(t**(-1)) # Representa a cardinalidade do conjunto t_n (ou em quantas partições irei discretizar)
    ## Up down Factors inicias
    u = factors[0]
    d = factors[1]

    udisc = (u)**(1/(ts-1)) # Discretização do Up factor
    ddisc = (d)**(1/(ts-1)) # Discretização do Down factor
    
    ## taxa de juros
    r = factors[2]

    ## probabilidades neutra-risco
    ## Definidas pela equação 1.2.15 do Teorema 1.2.2 p12 Shreeve
    ptil = 1/2
    qtil = 1/2

    ## rotina para garantir sanidade do modelo 0 < d < 1+r < u
    assert(0 < d and d < 1+r and 1+r < u)
    Sf = []
    for i in range(0,ts+1):
        Sf.append(udisc**(t-1-i)*ddisc**(i)*S[0])
    return Sf

In [43]:
discretizador([4,8,2],1/4)

[3.363585661014858,
 2.1189261887185906,
 1.3348398541700346,
 0.8408964152537147,
 0.5297315471796477]

In [36]:
def Sfinais(s0,Sf, t):
    ts = int(t**(-1)) # Representa a cardinalidade do conjunto t_n (ou em quantas partições irei discretizar)
    u = Sf[0]/s0 # Up factor geral
    d = Sf[1]/s0 # Down factor geral
    udisc = (Sf[0]/s0)**(1/(ts-1)) # Discretização do Up factor
    ddisc = (Sf[1]/s0)**(1/(ts-1)) # Discretização do Down factor
    S = []
    for i in range(0,ts):
        S.append(udisc**(t-1-i)*ddisc**(i)*s0)
    return S

In [37]:
Sfinais(4,[8,2],1/4)

[3.363585661014858, 2.1189261887185906, 1.3348398541700346, 0.8408964152537147]

In [38]:
8/4

2.0

In [39]:
2/4

0.5

In [125]:
def binomial(s0,sf,t,k,r,tipo=0):
    ## s0 valor inicial da binomial
    ## sf valores finais da binomial
    ## t fração a ser dividido o tempo

    ts = int(t**(-1))
    u = sf[0]/s0
    d = sf[1]/s0
    udisc = u**(1/(ts-1))
    ddisc = d**(1/(ts-1))
    def geraS():
        S = [[s0]]
        for i in range(1,ts):
            aux = []
            c = len(S[-1])
            for j in range(0,c+1):
                aux.append(s0*(udisc**(c-j))*(ddisc**(j)))
            S.append(aux)
        return S

    ## S-> Valores de Stock price
    ## k-> Valor do call/put
    ## tipo-> 0: call, 1: put

    def geraV(S):

        ## Metódo que dá um call/put nos valores finais
        def setaVf(sf):
            vf = []
            for i in range(len(sf)):
                if(tipo == 0):
                    if k >= sf[i]: vf.append((0,sf[i]))
                    else: vf.append((sf[i] - k,sf[i]))
                else:
                    if sf[i] >= k: vf.append(0)
                    else: vf.append(k - sf[i])
            return vf

        ## Algoritmo da seção 1.3
        def V(s,n):
            for i in range(0,n+1):
                if 
                vH = [i[0] if i[1] == udisc*s for i in vf]
            vT = [i[0] if i[1] == ddisc*s for i in vf]
            return (1/(1+r))*()

        return setaVf(S[-1])
    
    Ss = geraS()
    Vs = geraV(Ss)
    return [Ss,Vs]

SyntaxError: invalid syntax (<ipython-input-125-2c66f6a82bd4>, line 41)

In [121]:
S = geraS(4,[8,2],1/4)

In [122]:
geraV(S,5,1/4)

[(3.0, 8.0),
 (0.03968419957949365, 5.039684199579494),
 (0, 3.1748021039363996),
 (0, 2.0000000000000004)]

In [123]:
S[-2]

[6.349604207872798, 4.0, 2.519842099789747]